<a href="https://colab.research.google.com/github/weidafeng/ML/blob/master/face_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!df

Filesystem     1K-blocks    Used Available Use% Mounted on
overlay         41022688 7915296  30993856  21% /
tmpfs            6667604       0   6667604   0% /dev
tmpfs            6667604       0   6667604   0% /sys/fs/cgroup
tmpfs            6667604       4   6667600   1% /var/colab
/dev/sda1       47259264 9276940  37965940  20% /etc/hosts
shm                65536       0     65536   0% /dev/shm
tmpfs            6667604       0   6667604   0% /sys/firmware


In [0]:
# upload zip file of your data directory to colab using their (Google's) instructions.
from google.colab import files
# uploaded = files.upload()

In [0]:
# Once zip file is uploaded, perform the following operations:

import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded['face.zip']), "r")
zf.extractall()

# Your data directory should now be in colab's working directory under a 'data' directory.

In [4]:
!ls

face  face.zip	sample_data


In [5]:
!uzip faca.zip 

/bin/bash: uzip: command not found


In [6]:
!unzip -n face.zip -o face/

Archive:  face.zip
caution: filename not matched:  -o


In [7]:
!unzip -n face.zip -d face/

Archive:  face.zip
   creating: face/face/
  inflating: face/face/1.jpg         
  inflating: face/face/100.jpg       
  inflating: face/face/1001.jpg      
  inflating: face/face/1002.jpg      
  inflating: face/face/1003.jpg      
  inflating: face/face/1004.jpg      
  inflating: face/face/1005.jpg      
  inflating: face/face/1006.jpg      
  inflating: face/face/1007.jpg      
  inflating: face/face/1008.jpg      
  inflating: face/face/1009.jpg      
  inflating: face/face/101.jpg       
  inflating: face/face/1010.jpg      
  inflating: face/face/1011.jpg      
  inflating: face/face/1012.jpg      
  inflating: face/face/1013.jpg      
  inflating: face/face/1015.jpg      
  inflating: face/face/1016.jpg      
  inflating: face/face/1017.jpg      
  inflating: face/face/1018.jpg      
  inflating: face/face/1019.jpg      
  inflating: face/face/102.jpg       
  inflating: face/face/1020.jpg      
  inflating: face/face/1021.jpg      
  inflating: face/face/1022.jpg      
  infla

In [3]:
# !pip install imutils

from keras.layers import Input,Dense, Conv2D, MaxPooling2D,UpSampling2D
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import cv2
import os
from os import listdir

def image_to_feature_vector(img, size=(96,96)):         # Avg Dim 1798,1260
    return cv2.resize(img, size).flatten()
  

  
path = "face/face"

onlyfiles = [f for f in listdir(path)]
data = []
for f in onlyfiles:
    fp = os.path.join(path, f)
    x = cv2.imread(fp)
    x = image_to_feature_vector(x)
    x = x.astype('float32') / 255
    x = np.reshape(x, (96, 96, 3))
    data.append(x)
print(len(data))

6937


In [0]:
x_train = np.reshape(data[:5000], (5000, 96, 96, 3))
x_test = np.reshape(data[5001:], (1936, 96, 96, 3))

input_image = Input(shape=(96, 96, 3))

x = Conv2D(12, (3, 3), activation='relu', padding='same')(input_image)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(24, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(48, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

encoded = MaxPooling2D((2, 2), padding='same')(x)

# x = UpSampling2D((2, 2))(encoded)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(encoded)
# x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
x = Conv2D(48, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(24, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(12, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_image, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
autoencoder.summary()

# autoencoder.fit(x_train, x_train, epochs=100, batch_size=128,
#                     shuffle=True, validation_data=(x_test, x_test))

# 自定义的callback 每轮可视化一次
class Visual_Each_Epoch(Callback):

    def on_epoch_end(self, epoch ,logs={}):
#         self.losses.append(logs['loss'])
        self.show_and_save_image(epoch)
    
    def show_and_save_image(self,epoch):
    #     encoded_input = Input(shape=(1,1,32))
    #     decoder_layer = autoencoder.layers[-1]
    #     decoder = Model(encoded_input, decoder_layer(encoded_input) ) 

        decoded_imgs = autoencoder.predict(x_test[20:])

        n = 10
#         plt.title('Epoch %s'%(int(epoch)))
        plt.figure(figsize=(20, 4))
        for i in range(n):
            # display original
            ax = plt.subplot(2, n, i + 1)
            pic = cv2.cvtColor(x_test[i+20].reshape( 96,96,3), cv2.COLOR_BGR2RGB)
            plt.imshow(pic)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display encoded
        #     ax = plt.subplot(2, n, i + 1 + n)
        #     plt.imshow(encoded_img[i].reshape(3,3,128))
        #     plt.gray()
        #     ax.get_xaxis().set_visible(False)
        #     ax.get_yaxis().set_visible(False)

            # display reconstruction
            ax = plt.subplot(2, n, i + 1 + n)
            pic =cv2.cvtColor(decoded_imgs[i].reshape(96,96,3), cv2.COLOR_BGR2RGB)
    #         plt.savefig("%s.png"%n)
            plt.imshow(pic)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
#         plt.savefig("./face_logs_5/epoch%s.png"%epoch)
        plt.show()

#%%
visual_each_epoch = Visual_Each_Epoch()


# In[ ]:


# tensorboard
tbCallBack = TensorBoard(log_dir='./face_logs_6',histogram_freq=1,write_graph=True,write_images=True)

# checkpoint
filepath="./face_logs_6/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpointer = ModelCheckpoint(filepath=filepath , monitor='val_acc', mode='auto' ,save_best_only=True ,verbose=1)
# checkpointer = ModelCheckpoint(filepath="./face_logs/weights.hdf5", verbose=1, save_best_only=False)

# learning rate adjust dynamic
# lrate = ReduceLROnPlateau(min_lr=0.00001)

# json_string = autoencoder.to_json( )  #等价于 json_string = model.get_config()
# open('./face_logs_6/my_model_architecture.json','w').write(json_string)  

autoencoder.fit(x_train, x_train, epochs=30, batch_size=32,
                    shuffle=True, validation_data=(x_test, x_test),callbacks=[tbCallBack, checkpointer ,visual_each_epoch])



In [22]:
!ls
!cd face_logs_6/

# files.download('face_logs_6')

face  face_logs_6  face_logs_6_colab_30.zip  face.zip  sample_data


In [24]:
tensorboard --logdir=face_logs_6

SyntaxError: ignored

In [0]:
# !zip -r face_logs_6_colab_30.zip face_logs_6/
files.download('face_logs_6_colab_30.zip')

In [0]:

# coding: utf-8

# In[1]:


# import the labiary
from keras.layers import Input,Dense, Conv2D, MaxPooling2D,UpSampling2D
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau ,TensorBoard, Callback
# from keras import backend as K #方便后面自定义损失函数

from keras.models import load_model,model_from_json

# sub_pixel_conv
import tensorflow as tf
from keras.layers import Lambda
from keras import backend as K

# from keras.callbacks import TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt

from imutils import paths
import numpy as np
import argparse
import cv2
import os
from os import listdir


# In[2]:


def _phase_shift(I, r):
    input_shape = tf.shape(I)
    bsize = input_shape[0]
    a = input_shape[1]
    b = input_shape[2]
    c = input_shape[3]
    shape = tf.stack([bsize,a,b,r,r])

    input_shape_as_numbers = I.get_shape()

    X = tf.reshape(tensor=I, shape=shape)
    X = tf.transpose(X, (0, 1, 2, 4, 3))  # bsize, a, b, 1, 1
    X = tf.split(X, input_shape_as_numbers[1], axis=1)  # a, [bsize, b, r, r]
    X = tf.concat([tf.squeeze(x) for x in X], axis=2)  # bsize, b, a*r, r
    X = tf.split(X,input_shape_as_numbers[2], axis=1)  # b, [bsize, a*r, r]
    X = tf.concat([tf.squeeze(x) for x in X], axis=2)  # bsize, a*r, b*r
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def PS(input_shape, r, color=False):

	def subpixel_shape(input_shape):
		dims = [input_shape[0],
			input_shape[1] * r,
			input_shape[2] * r,
			int(input_shape[3] / (r ** 2))]
		output_shape = tuple(dims)
		return output_shape
  
	def subpixel(x):
		if color:
			Xc = tf.split(3, 3, X)
			X = tf.concat(3, [_phase_shift(x, r) for x in Xc])
		else:
			x_upsampled = _phase_shift(x, r)
		return x_upsampled

	return Lambda(subpixel, output_shape=subpixel_shape, name='subpixel')


# In[3]:

class Subpixel(Conv2D):
    def __init__(self,
                 filters,
                 kernel_size,
                 r,
                 padding='valid',
                 data_format=None,
                 strides=(1,1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(Subpixel, self).__init__(
            filters=r*r*filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)
        self.r = r

    def _phase_shift(self, I):
        r = self.r
        bsize, a, b, c = I.get_shape().as_list()
        bsize = K.shape(I)[0] # Handling Dimension(None) type for undefined batch dim
        X = K.reshape(I, [bsize, a, b, c/(r*r),r, r]) # bsize, a, b, c/(r*r), r, r
        X = K.permute_dimensions(X, (0, 1, 2, 5, 4, 3))  # bsize, a, b, r, r, c/(r*r)
        #Keras backend does not support tf.split, so in future versions this could be nicer
        X = [X[:,i,:,:,:,:] for i in range(a)] # a, [bsize, b, r, r, c/(r*r)
        X = K.concatenate(X, 2)  # bsize, b, a*r, r, c/(r*r)
        X = [X[:,i,:,:,:] for i in range(b)] # b, [bsize, r, r, c/(r*r)
        X = K.concatenate(X, 2)  # bsize, a*r, b*r, c/(r*r)
        return X

    def call(self, inputs):
        return self._phase_shift(super(Subpixel, self).call(inputs))

    def compute_output_shape(self, input_shape):
        unshifted = super(Subpixel, self).compute_output_shape(input_shape)
        return (unshifted[0], self.r*unshifted[1], self.r*unshifted[2], unshifted[3]/(self.r*self.r))

    def get_config(self):
        config = super(Conv2D, self).get_config()
        config.pop('rank')
        config.pop('dilation_rate')
        config['filters']/=self.r*self.r
        config['r'] = self.r
        return config


# image path
path ='face/face'


#resize and flatten image to 1-d array([img_width*img_height*img_channel,1])
def image_to_feature_vector(img, size=(96,96)):         # Avg Dim 1798,1260
    return cv2.resize(img, size).flatten()

#get the image_file name
#e.g. 012.jpg,013.jpg
image_file_name = [f for f in listdir(path)]

#stack and store image_data in memory
data = []
for f in image_file_name:
    fp = os.path.join(path, f)  #generate the abs_path of the images,e.g. 'f:/image_path/012.jpg'
    x = cv2.imread(fp)
    x = image_to_feature_vector(x) # faltten
    x = x.astype('float32') / 255  # normalization ,express as a floating point number from 0 to 1.
    x = np.reshape(x, (96, 96, 3)) # reshape  
    data.append(x)
print('The number of train & text image: ',len(data))


# In[4]:


# split the dataset to train and test
x_train = np.reshape(data[:5000], (5000, 96, 96, 3))
x_test = np.reshape(data[5001:], (1936, 96, 96, 3)) #1936

# # split the dataset to train and test
# x_train = np.reshape(data[:500], (500, 96, 96, 3))
# x_test = np.reshape(data[501:], (55, 96, 96, 3)) #1936


# In[5]:


# define the encoder model 

input_image = Input(shape=(96, 96, 3))

x = Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')(input_image)
# x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2), padding='same')(x)
# "encoder" is the encodered data 

encoded = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)

# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

# encoded = MaxPooling2D((2, 2), padding='same')(x)


# In[6]:


# x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(encoded)
# #x = PS(x.shape, 3, color=False)(x)
# x = Subpixel(3, (3,3), 2, activation='relu',padding='same')(x)

# x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = Subpixel(3, (3,3), 2, activation='relu',padding='same')(x)
# #x = PS(x.shape, 3, color=False)(x)

# x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = Subpixel(3, (3,3), 2, activation='relu',padding='same')(x)
# #x = PS(x.shape, 3, color=False)(x)


# #x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# # decoded = Subpixel(3, (3,3), 2,activation='relu')(x)
# #x = PS(x.shape, 3, color=False)(x)
# decoded = Conv2D(3, (3, 3), strides=(1, 1), activation='sigmoid', padding='same')(x) # note: back to 3-dimention




# x = UpSampling2D((2, 2))(encoded)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(encoded)
# x = Conv2D(96, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(encoded)
x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
# x = Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
x = Subpixel(3, (3,3), 2, activation='relu',padding='same')(x)
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)

# # I have a question: should the decode model depth from low to high or the other?
decoded = Conv2D(3, (3, 3), strides=(1, 1), activation='sigmoid', padding='same')(x) # note: back to 3-dimention
# # decoded = UpSampling2D((2,2))(x)


# In[7]:


'''
# 自定义loss为预测值与真实值的差
def mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), np.inf) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), np.inf) + 1.)
    return K.mean(K.square(first_log - second_log), axis=-1)
    
def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.mean(y_true * y_pred, axis=-1)
'''

autoencoder = Model(input_image, decoded) # use the encoder&decoder model to train
autoencoder.compile(optimizer='adadelta', loss='mse', metrics=['acc']) 
autoencoder.summary()


# In[8]:


# 自定义的callback 每轮可视化一次
class Visual_Each_Epoch(Callback):

    def on_epoch_end(self, epoch ,logs={}):
#         self.losses.append(logs['loss'])
        self.show_and_save_image(epoch)
    
    def show_and_save_image(self,epoch):
    #     encoded_input = Input(shape=(1,1,32))
    #     decoder_layer = autoencoder.layers[-1]
    #     decoder = Model(encoded_input, decoder_layer(encoded_input) ) 

        decoded_imgs = autoencoder.predict(x_test[20:])

        n = 10
#         plt.title('Epoch %s'%(int(epoch)))
        plt.figure(figsize=(20, 4))
        for i in range(n):
            # display original
            ax = plt.subplot(2, n, i + 1)
            pic = cv2.cvtColor(x_test[i+20].reshape( 96,96,3), cv2.COLOR_BGR2RGB)
            plt.imshow(pic)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display encoded
        #     ax = plt.subplot(2, n, i + 1 + n)
        #     plt.imshow(encoded_img[i].reshape(3,3,128))
        #     plt.gray()
        #     ax.get_xaxis().set_visible(False)
        #     ax.get_yaxis().set_visible(False)

            # display reconstruction
            ax = plt.subplot(2, n, i + 1 + n)
            pic =cv2.cvtColor(decoded_imgs[i].reshape(96,96,3), cv2.COLOR_BGR2RGB)
    #         plt.savefig("%s.png"%n)
            plt.imshow(pic)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        plt.savefig("./face_logs_5/epoch%s.png"%epoch)
        plt.show()

#%%
visual_each_epoch = Visual_Each_Epoch()


# In[ ]:


# tensorboard
tbCallBack = TensorBoard(log_dir='./face_logs_6',histogram_freq=1,write_graph=True,write_images=True)

# checkpoint
filepath="./face_logs_6/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpointer = ModelCheckpoint(filepath=filepath , monitor='val_acc', mode='auto' ,save_best_only=True ,verbose=1)
# checkpointer = ModelCheckpoint(filepath="./face_logs/weights.hdf5", verbose=1, save_best_only=False)

# learning rate adjust dynamic
# lrate = ReduceLROnPlateau(min_lr=0.00001)

# json_string = autoencoder.to_json( )  #等价于 json_string = model.get_config()
# open('./face_logs_6/my_model_architecture.json','w').write(json_string)  

autoencoder.fit(x_train, x_train, epochs=30, batch_size=32,
                    shuffle=True, validation_data=(x_test, x_test),callbacks=[tbCallBack, checkpointer ,visual_each_epoch])


# autoencoder.save_weights('./face_logs_5my_model_weights.h5')  


# In[ ]:


# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

In [0]:
!mkdir face_logs_5